# Fifa World Cup 2018 - Prediction

## Brainstorming

This section is dedicated to the Exploratory Data Analysis and brainstoriming on possible faetures and problem formulation.

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy
%matplotlib inline

### Dataset 1: [International football results from 1872 to 2018](https://www.kaggle.com/martj42/international-football-results-from-1872-to-2017/data):

This data set contains the results of internation games for the past 146 years.

AG: Let's import the main data set(international games).
With this we can get some data for all the available games till now.
Especially the last 2 decades of these data can be used to extract some useful information for each team. These are the features I think we can get out of this data:

- Win to lose ratio total
- Win to lose ration general
- Mean goal scored ( **this can be used to define weather team is defensive or not** )
- Mean goal recieved ( **this can be used to define weather team is offensive or not** )

In [39]:
international_matches = pd.read_csv('international_matches.zip', compression='zip', header=0, sep=',', quotechar='"')

In [40]:
international_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38902 entries, 0 to 38901
Data columns (total 8 columns):
date          38902 non-null object
home_team     38902 non-null object
away_team     38902 non-null object
home_score    38902 non-null int64
away_score    38902 non-null int64
tournament    38902 non-null object
city          38902 non-null object
country       38902 non-null object
dtypes: int64(2), object(6)
memory usage: 2.4+ MB


 ### Initial content's dataset

In [41]:
international_matches.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland
1,1873-03-08,England,Scotland,4,2,Friendly,London,England
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland
3,1875-03-06,England,Scotland,2,2,Friendly,London,England
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland


In [42]:
def process_team(row,type_team_prefix='home'):
    
    contrary_type = {
        "home": "away",
        "away": "home"
    }
    
    # Creating variables to handle 'home' and 'away' teams independently
    type_team = type_team_prefix + '_team'
    contrary_team = contrary_type.get(type_team_prefix) + '_team'
    
    type_team_score = type_team_prefix + '_score'
    contrary_team_score = contrary_type.get(type_team_prefix) + '_score'
    
    current_team = row[type_team]
    
    victory = row[type_team_score] > row[contrary_team_score]
    lose = row[type_team_score] < row[contrary_team_score]
    tie = row[type_team_score] == row[contrary_team_score]
    
    goals_scored = row[type_team_score]
    goals_received = row[contrary_team_score]
    
    if current_team not in all_series:
        
        serie_team = {
            "Team": current_team, 
            "Victories": int(victory), 
            "Loses": int(lose), 
            "Ties": int(tie), 
            "Ratio(win/lose)": 0, 
            "Goals(scored)": goals_scored, 
            "Goals(received)": goals_received
        }
        
        all_series[current_team] = serie_team
        
    else:
        
        if victory:
            all_series[current_team]['Victories'] += 1
        elif lose:
            all_series[current_team]['Loses'] += 1
        else:
            all_series[current_team]['Ties'] += 1
            
        all_series[current_team]['Goals(scored)'] += goals_scored
        all_series[current_team]['Goals(received)'] += goals_received

In [43]:
all_series = {}

for i , row in international_matches.iterrows():
    process_team(row,'home')
    process_team(row,'away')

In [44]:
columns = ["Team","Victories","Loses","Ties","Ratio(win/lose)", "Goals(scored)","Goals(received)"]

infos_teams = pd.DataFrame(list(all_series.values()),columns=columns)
infos_teams.head()

,Team,Victories,Loses,Ties,Ratio(win/lose),Goals(scored),Goals(received)
0,Scotland,361,233,165,0,1318,927
1,England,552,185,238,0,2132,961
2,Wales,197,301,138,0,799,1063
3,Northern Ireland,158,322,144,0,656,1188
4,USA,280,239,140,0,947,893


In [45]:
infos_teams['Ratio(win/lose)'] = infos_teams.Victories / infos_teams.Loses
infos_teams.sort_values('Ratio(win/lose)', ascending = False).head(10)
infos_teams['total_games'] = infos_teams.Victories + infos_teams.Loses + infos_teams.Ties

> Now we will normalize the columns. The reason is that some teams play much more than other and number of victories itself cannot show their strength. **

In [46]:
infos_teams.head()
infos_teams.columns
normalizing_columns = ['Victories', 'Loses', 'Ties',
       'Goals(scored)', 'Goals(received)'] # thse columns need to be normalized

for column in normalizing_columns:
    infos_teams[column] = round(infos_teams[column] / infos_teams['total_games'],4)

In [47]:
infos_teams.sort_values(['Ratio(win/lose)'],ascending=False).head(15)

,Team,Victories,Loses,Ties,Ratio(win/lose),Goals(scored),Goals(received),total_games
224,Jersey,0.5714,0.1429,0.2857,4.000000,1.5238,1.0476,21
24,Brazil,0.6363,0.1622,0.2015,3.921569,2.1877,0.9226,943
34,Spain,0.5823,0.1901,0.2275,3.062992,1.9865,0.9027,668
1,England,0.5662,0.1897,0.2441,2.983784,2.1867,0.9856,975
15,Germany,0.5862,0.2080,0.2058,2.818653,2.2435,1.1541,928
82,Croatia,0.5358,0.1945,0.2696,2.754386,1.7474,0.9522,293
86,Iran,0.5432,0.1979,0.2589,2.744681,1.8442,0.8147,475
18,Italy,0.5256,0.1931,0.2813,2.721854,1.6982,0.9757,782
7,Argentina,0.5391,0.2135,0.2474,2.524752,1.8721,1.0455,946
101,Korea Republic,0.5265,0.2169,0.2566,2.427778,1.7687,0.9108,830


> Now we will add the name of the contients to the columns. **

In [48]:
country_continent = pd.read_csv('Countries_Continents.csv')
country_continent[country_continent.Team == 'Scotland']

,Continent,Team
122,Europe,Scotland


In [37]:
infos_teams = infos_teams.merge(country_continent, on ='Team', how = 'left').sort_values(['Ratio(win/lose)'],ascending=False)
infos_teams.sort_values(['Ratio(win/lose)'],ascending=False).head(15)

,Team,Victories,Loses,Ties,Ratio(win/lose),Goals(scored),Goals(received),total_games,Continent
224,Jersey,0.5714,0.1429,0.2857,4.000000,1.5238,1.0476,21,NaN
24,Brazil,0.6363,0.1622,0.2015,3.921569,2.1877,0.9226,943,South America
34,Spain,0.5823,0.1901,0.2275,3.062992,1.9865,0.9027,668,Europe
1,England,0.5662,0.1897,0.2441,2.983784,2.1867,0.9856,975,NaN
15,Germany,0.5862,0.2080,0.2058,2.818653,2.2435,1.1541,928,Europe
82,Croatia,0.5358,0.1945,0.2696,2.754386,1.7474,0.9522,293,Europe
86,Iran,0.5432,0.1979,0.2589,2.744681,1.8442,0.8147,475,Asia
18,Italy,0.5256,0.1931,0.2813,2.721854,1.6982,0.9757,782,Europe
7,Argentina,0.5391,0.2135,0.2474,2.524752,1.8721,1.0455,946,South America
101,Korea Republic,0.5265,0.2169,0.2566,2.427778,1.7687,0.9108,830,NaN


> Now we will test the results generated. We use the ```Victory``` attribute from **England** and **USA **

In [10]:
victory_england_as_home_team = (international_matches.home_team == 'England') & (international_matches.home_score > international_matches.away_score)
victory_england_as_away_team = (international_matches.away_team == 'England') & (international_matches.away_score > international_matches.home_score)

victory_usa_as_home_team = (international_matches.home_team == 'USA') & (international_matches.home_score > international_matches.away_score)
victory_usa_as_away_team = (international_matches.away_team == 'USA') & (international_matches.away_score > international_matches.home_score)

In [11]:
print("Victories England: {}".format(len(international_matches[victory_england_as_home_team | victory_england_as_away_team])))
print("Victories USA: {}".format(len(international_matches[victory_usa_as_home_team | victory_usa_as_away_team])))

Victories England: 552
Victories USA: 280


### Yes!! We did process the initial file to find each attribute's team to discover your victories, ties and loses as well as your goals scored and received.

### Dataset 2: fifa 2018 schedule

> This dataset consist of all the games in the upcoming matches. What we are thinking of getting from this data is the name of the participating teams. Moreover, eventually we use them to predict all of the games and the champion.

In [12]:
fifa2018 = pd.read_csv('fifa2018.csv')

In [13]:
counteris2018 = fifa2018['Home Team'].unique()[:32]

In [14]:
counteris2018

array(['Russia', 'Egypt', 'Morocco', 'Portugal', 'France', 'Argentina',
       'Peru', 'Croatia', 'Costa Rica', 'Germany', 'Brazil', 'Sweden',
       'Belgium', 'Tunisia', 'Colombia', 'Poland', 'Uruguay', 'Iran',
       'Denmark', 'Nigeria', 'Serbia', 'Korea Republic', 'England',
       'Japan', 'Saudi Arabia', 'Spain', 'Australia', 'Iceland', 'Mexico',
       'Switzerland', 'Senegal', 'Panama'], dtype=object)

### Dataset 3: [European Soccer Database](https://www.kaggle.com/hugomathien/soccer/data):

(can anybody add some explanation here?)
!!! You need to extract the players.sqlite.zip to the same folder !!!

In [15]:
import sqlite3
uri_database = "sqlite:///database.sqlite"

In [16]:
players = pd.read_sql_table("Player",uri_database)
players_attributes = pd.read_sql_table("Player_Attributes",uri_database)
team_attributes = pd.read_sql_table("Team_Attributes", uri_database)

In [17]:
print("Players DF: ", players.shape)
print("Players attributes DF: ", players_attributes.shape)
print("Team attributes DF: ", team_attributes.shape)

Players DF:  (11060, 7)
Players attributes DF:  (183978, 42)
Team attributes DF:  (1458, 25)


## EDA
Can someone do some EDA?